In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'
# mol_name = 'H2'

# mol_name = 'LiH'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [5]:
from symred.chem import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="stick")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
from symred.chem import PySCFDriver

In [12]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = False,
run_ccsd = False,
run_fci  = False

In [13]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [14]:
pyscf_obj.run_pyscf()

In [15]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [16]:
from symred.chem import FermionicHamilt

In [17]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 2.29310124732
((0, 1), (0, 0)) -1.8385126260957712
((0, 1), (4, 0)) 0.1535200531800066
((1, 1), (1, 0)) -1.8385126260957712
((1, 1), (5, 0)) 0.1535200531800066
((2, 1), (2, 0)) -1.5485198738611377
((2, 1), (6, 0)) 0.1346528866850458
((3, 1), (3, 0)) -1.5485198738611377
((3, 1), (7, 0)) 0.1346528866850458
((4, 1), (0, 0)) 0.15352005318000683
((4, 1), (4, 0)) -1.2423978239696827
((5, 1), (1, 0)) 0.15352005318000683
((5, 1), (5, 0)) -1.2423978239696827
((6, 1), (2, 0)) 0.13465288668504574
((6, 1), (6, 0)) -0.9084576464931442
((7, 1), (3, 0)) 0.13465288668504574
((7, 1), (7, 0)) -0.9084576464931442
((0, 1), (0, 1), (0, 0), (0, 0)) 0.2504327702741458
((0, 1), (0, 1), (0, 0), (4, 0)) -0.04165187456702536
((0, 1), (0, 1), (2, 0), (2, 0)) 0.07725837026056045
((0, 1), (0, 1), (2, 0), (6, 0)) -0.02286714657234941
((0, 1), (0, 1), (4, 0), (0, 0)) -0.041651874567025335
((0, 1), (0, 1), (4, 0), (4, 0)) 0.053084040051590224
((0, 1), (0, 1), (6, 0), (2, 0)) -0.022867146572349395
((0, 1), (0, 1), (

In [19]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [20]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(256,)

In [21]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-2.0982818585784786
-2.0982818585784786


(-2.0982818585784786+0j)

In [22]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
e_orbs_occ = fij[:nocc]
e_i = e_orbs_occ.reshape(1, 1, -1, 1)
e_j = e_orbs_occ.reshape(1, 1, 1, -1)

e_orbs_vir = fij[nocc:]
e_a = e_orbs_vir.reshape(-1, 1, 1, 1)
e_b = e_orbs_vir.reshape(1, -1, 1, 1)

abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (e_i + e_j - e_a - e_b)

## same spin - different spins
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.04160571461928189

In [23]:
pyscf_obj.pyscf_mp2.e_corr

-0.04160571461928188

In [25]:
from pyscf.cc.addons import spatial2spin

t2 = spatial2spin(pyscf_obj.pyscf_mp2.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [26]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])

print(len(double_amplitudes_list))
double_amplitudes_list

40


[[[4, 0, 5, 1], -0.028914558866912243],
 [[4, 0, 6, 2], -0.010778158725408358],
 [[4, 0, 7, 3], -0.022405585894956476],
 [[4, 1, 5, 0], 0.028914558866912243],
 [[4, 1, 7, 2], 0.011627427169548121],
 [[4, 2, 5, 3], -0.05280759309647757],
 [[4, 2, 6, 0], 0.010778158725408347],
 [[4, 2, 7, 1], -0.011627427169548121],
 [[4, 3, 5, 2], 0.05280759309647757],
 [[4, 3, 7, 0], 0.022405585894956476],
 [[5, 0, 4, 1], 0.028914558866912243],
 [[5, 0, 6, 3], 0.011627427169548118],
 [[5, 1, 4, 0], -0.028914558866912243],
 [[5, 1, 6, 2], -0.02240558589495647],
 [[5, 1, 7, 3], -0.010778158725408358],
 [[5, 2, 4, 3], 0.05280759309647757],
 [[5, 2, 6, 1], 0.02240558589495647],
 [[5, 3, 4, 2], -0.05280759309647757],
 [[5, 3, 6, 0], -0.011627427169548118],
 [[5, 3, 7, 1], 0.010778158725408347],
 [[6, 0, 4, 2], 0.010778158725408358],
 [[6, 0, 5, 3], -0.011627427169548118],
 [[6, 0, 7, 1], -0.016527871022874608],
 [[6, 1, 5, 2], 0.02240558589495647],
 [[6, 1, 7, 0], 0.016527871022874608],
 [[6, 2, 4, 0], -0.0

In [28]:
generator = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [181]:
len(list(generator))

40

In [34]:
mp2_ansatz = expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [35]:
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-2.1398875731977607+0j)

In [37]:
pyscf_obj.pyscf_mp2.e_tot

-2.1398875731977607

In [91]:
(mp2_ket.conj().T  @ hf_ket) # "intermediate normalization!"

(1+0j)

In [38]:
overlap_mp2 = (mp2_ket.conj().T @ mp2_ket).real
C0 = 1/np.sqrt(overlap_mp2)

norm_state = (mp2_ket) * C0

norm_state.conj().T @ norm_state

(1.0000000000000002+0j)

In [49]:
# shouldn't use this H_mat (as it contains more terms than MP2 considers.)
(norm_state.conj().T @ H_mat @ norm_state).real

-2.1549414883264264

In [51]:
norm_state.conj().T @ hf_ket

(0.9884912786489918+0j)